In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
independent=dataset[["Age","EstimatedSalary","Gender_Male"]]
dependent=dataset["Purchased"]

In [9]:
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,True
1,35,20000,True
2,26,43000,False
3,27,57000,False
4,19,76000,True
...,...,...,...
395,46,41000,False
396,51,23000,True
397,50,20000,False
398,36,33000,True


In [10]:
dependent

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [11]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(independent, dependent, test_size = 1/3, random_state = 0)

In [12]:
from sklearn.naive_bayes import ComplementNB

from sklearn.model_selection import GridSearchCV

param_grid = {
      'alpha': [0.1, 0.5, 1.0, 10],           
    'fit_prior': [True, False],              
    'class_prior': [None],                  
    'norm': [True, False]  
    }

grid = GridSearchCV(ComplementNB(), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')

# Fit the model
grid.fit(X_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(estimator=ComplementNB(), n_jobs=-1,
             param_grid={'alpha': [0.1, 0.5, 1.0, 10], 'class_prior': [None],
                         'fit_prior': [True, False], 'norm': [True, False]},
             scoring='f1_weighted', verbose=3)

In [13]:
re=grid.cv_results_

grid_predictions = grid.predict(X_test) 
   
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [14]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'alpha': 0.1, 'class_prior': None, 'fit_prior': True, 'norm': False}: 0.5224047968734963


In [15]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[40 45]
 [20 29]]


In [16]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.67      0.47      0.55        85
           1       0.39      0.59      0.47        49

    accuracy                           0.51       134
   macro avg       0.53      0.53      0.51       134
weighted avg       0.57      0.51      0.52       134



In [17]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.5834333733493396

In [18]:
table=pd.DataFrame.from_dict(re)

In [19]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_class_prior,param_fit_prior,param_norm,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.010450,0.002914,0.025171,0.002837,0.1,None,True,True,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.183156,0.172203,0.189203,0.189203,0.189203,0.184594,0.006623,9
1,0.008730,0.001696,0.019282,0.008370,0.1,None,True,False,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.509839,0.614012,0.563318,0.482156,0.556133,0.545092,0.045624,1
2,0.008635,0.001574,0.009353,0.000390,0.1,None,False,True,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.183156,0.172203,0.189203,0.189203,0.189203,0.184594,0.006623,9
3,0.008387,0.001022,0.010710,0.001801,0.1,None,False,False,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.509839,0.614012,0.563318,0.482156,0.556133,0.545092,0.045624,1
4,0.009937,0.002298,0.010741,0.001846,0.5,None,True,True,"{'alpha': 0.5, 'class_prior': None, 'fit_prior...",0.183156,0.172203,0.189203,0.189203,0.189203,0.184594,0.006623,9
5,0.008686,0.001575,0.008284,0.000531,0.5,None,True,False,"{'alpha': 0.5, 'class_prior': None, 'fit_prior...",0.509839,0.614012,0.563318,0.482156,0.556133,0.545092,0.045624,1
6,0.007558,0.000885,0.009823,0.000671,0.5,None,False,True,"{'alpha': 0.5, 'class_prior': None, 'fit_prior...",0.183156,0.172203,0.189203,0.189203,0.189203,0.184594,0.006623,9
7,0.007749,0.000715,0.010446,0.002243,0.5,None,False,False,"{'alpha': 0.5, 'class_prior': None, 'fit_prior...",0.509839,0.614012,0.563318,0.482156,0.556133,0.545092,0.045624,1
8,0.008141,0.001271,0.009008,0.001079,1.0,None,True,True,"{'alpha': 1.0, 'class_prior': None, 'fit_prior...",0.183156,0.172203,0.189203,0.189203,0.189203,0.184594,0.006623,9
9,0.008281,0.002469,0.009202,0.003599,1.0,None,True,False,"{'alpha': 1.0, 'class_prior': None, 'fit_prior...",0.509839,0.614012,0.563318,0.482156,0.556133,0.545092,0.045624,1


In [20]:
age_input=float(input("Age:"))
salary_input=float(input("Salary:"))
sex_male_input=int(input("Sex Male 0 or 1:"))

Age:43
Salary:30000
Sex Male 0 or 1:0


In [21]:
Future_Prediction=grid.predict([[age_input,salary_input,sex_male_input]]) 
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]


C:\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but ComplementNB was fitted with feature names
  warnings.warn(
